In [92]:
html = '''{% if refund_line_items.size == item_count %}
  {% capture email_title %}Your order has been refunded{% endcapture %}
{% elsif refund_line_items.size == 0 %}
  {% capture email_title %}You have received a refund{% endcapture %}
{% else %}
  {% capture email_title %}Some items in your order have been refunded{% endcapture %}
{% endif %}
{% capture email_body %}Total amount refunded: <strong>{{ amount | money_with_currency }}</strong>. It may take up to 10 days for this refund to appear in your account.{% endcapture %}

<!DOCTYPE html>
<html lang="en">
  <head>
  <title>{{ email_title }}</title>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
  <meta name="viewport" content="width=device-width">
  <link rel="stylesheet" type="text/css" href="/assets/notifications/styles.css">
  <style>
    .button__cell { background: {{ shop.email_accent_color }}; }
    a, a:hover, a:active, a:visited { color: {{ shop.email_accent_color }}; }
  </style>
</head>

  <body>
    <table class="body">
      <tr>
        <td>
          <table class="header row">
  <tr>
    <td class="header__cell">
      <center>

        <table class="container">
          <tr>
            <td>

              <table class="row">
                <tr>
                  <td class="shop-name__cell">
                    {%- if shop.email_logo_url %}
                      <img src="{{shop.email_logo_url}}" alt="{{ shop.name }}" width="{{ shop.email_logo_width }}">
                    {%- else %}
                      <h1 class="shop-name__text">
                        <a href="{{shop.url}}">{{ shop.name }}</a>
                      </h1>
                    {%- endif %}
                  </td>

                    <td>
                      <tr>
                        <td class="order-number__cell">
                          <span class="order-number__text">
                            Order {{ order_name }}
                          </span>
                        </td>
                      </tr>
                      {%- if po_number %}
                          <tr>
                            <td class="po-number__cell">
                              <span class="po-number__text">
                                PO number #{{ po_number }}
                              </span>
                            </td>
                          </tr>
                      {%- endif %}
                    </td>
                </tr>
              </table>

            </td>
          </tr>
        </table>

      </center>
    </td>
  </tr>
</table>

          <table class="row content">
  <tr>
    <td class="content__cell">
      <center>
        <table class="container">
          <tr>
            <td>
              
            <h2>{{ email_title }}</h2>
            <p>{{ email_body }}</p>

            </td>
          </tr>
        </table>
      </center>
    </td>
  </tr>
</table>

          <table class="row section">
  <tr>
    <td class="section__cell">
      <center>
        <table class="container">
          <tr>
            <td>
              <h3>Order summary</h3>
            </td>
          </tr>
        </table>
        <table class="container">
          <tr>
            <td>
              
            
<table class="row">
  {% for line in subtotal_line_items %}
  <tr class="order-list__item">
    <td class="order-list__item__cell">
      <table>
        <td>
          {% if line.image %}
            <img src="{{ line | img_url: 'compact_cropped' }}" align="left" width="60" height="60" class="order-list__product-image"/>
          {% endif %}
        </td>
        <td class="order-list__product-description-cell">
          {% if line.product.title %}
            {% assign line_title = line.product.title %}
          {% else %}
            {% assign line_title = line.title %}
          {% endif %}

          {% if line.quantity < line.quantity %}
            {% capture line_display %} {{ line.quantity }} of {{ line.quantity }} {% endcapture %}
          {% else %}
            {% assign line_display = line.quantity  %}
          {% endif %}

          <span class="order-list__item-title">{{ line_title }}&nbsp;&times;&nbsp;{{ line_display }}</span><br/>

          {% if line.variant.title != 'Default Title' %}
            <span class="order-list__item-variant">{{ line.variant.title }}</span><br/>
          {% endif %}

          {% for group in line.groups %}
            <span class="order-list__item-variant">Part of: {{ group.display_title }}</span><br/>
          {% endfor %}

            {% if line.gift_card and line.properties["__shopify_send_gift_card_to_recipient"] %}
              {% for property in line.properties %}
  {% assign property_first_char = property.first | slice: 0 %}
  {% if property.last != blank and property_first_char != '_' %}
    <div class="order-list__item-property">
      <dt>{{ property.first }}:</dt>
      <dd>
      {% if property.last contains '/uploads/' %}
        <a href="{{ property.last }}" class="link" target="_blank">
        {{ property.last | split: '/' | last }}
        </a>
      {% else %}
        {{ property.last }}
      {% endif %}
      </dd>
    </div>
  {% endif %}
{% endfor %}

            {% endif %}

          {% if line.selling_plan_allocation %}
            <span class="order-list__item-variant">{{ line.selling_plan_allocation.selling_plan.name }}</span><br/>
          {% endif %}

          {% if line.refunded_quantity > 0 %}
            <span class="order-list__item-refunded">Refunded</span>
          {% endif %}

          {% if line.discount_allocations %}
            {% for discount_allocation in line.discount_allocations %}
              {% if discount_allocation.discount_application.target_selection != 'all' %}
              <p>
                <span class="order-list__item-discount-allocation">
                  <img src="{{ 'notifications/discounttag.png' | shopify_asset_url }}" width="18" height="18" class="discount-tag-icon" />
                  <span>
                    {{ discount_allocation.discount_application.title | upcase }}
                    (-{{ discount_allocation.amount | money }})
                  </span>
                </span>
              </p>
              {% endif %}
            {% endfor %}
          {% endif %}
        </td>
          <td class="order-list__price-cell">
            {% if line.original_line_price != line.final_line_price %}
              <del class="order-list__item-original-price">{{ line.original_line_price | money }}</del>
            {% endif %}
            <p class="order-list__item-price">
              {% if line.final_line_price > 0 %}
                {{ line.final_line_price | money }}
              {% else %}
                Free
              {% endif %}
            </p>
          </td>
      </table>
    </td>
  </tr>{% endfor %}
</table>

            <table class="row subtotal-lines">
  <tr>
    <td class="subtotal-spacer"></td>
    <td>
      <table class="row subtotal-table">

        
{% assign order_discount_count = 0 %}
{% assign total_order_discount_amount = 0 %}
{% assign has_shipping_discount = false %}

{% for discount_application in discount_applications %}
  {% if discount_application.target_selection == 'all' and discount_application.target_type == 'line_item' %}
    {% assign order_discount_count = order_discount_count | plus: 1 %}
    {% assign total_order_discount_amount = total_order_discount_amount | plus: discount_application.total_allocated_amount  %}
  {% endif %}
  {% if discount_application.target_type == 'shipping_line' %}
    {% assign has_shipping_discount = true %}
    {% assign shipping_discount = discount_application.title %}
    {% assign shipping_amount = discount_application.total_allocated_amount %}
  {% endif %}
{% endfor %}



<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Subtotal</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ subtotal_price | plus: total_order_discount_amount | money }}</strong>
  </td>
</tr>



{% if order_discount_count > 0 %}
  {% if order_discount_count == 1 %}
    
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Order Discount</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>-{{ total_order_discount_amount | money }}</strong>
  </td>
</tr>

  {% endif %}
  {% if order_discount_count > 1 %}
    
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Order Discounts</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>-{{ total_order_discount_amount | money }}</strong>
  </td>
</tr>

  {% endif %}
  {% for discount_application in discount_applications %}
    {% if discount_application.target_selection == 'all' and discount_application.target_type != 'shipping_line' %}
      <tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span class="subtotal-line__discount">
        <img src="{{ 'notifications/discounttag.png' | shopify_asset_url }}" width="18" height="18" class="discount-tag-icon" />
        <span class="subtotal-line__discount-title">{{ discount_application.title }} (-{{ discount_application.total_allocated_amount | money }})</span>
      </span>
    </p>
  </td>
</tr>

    {% endif %}
  {% endfor %}
{% endif %}


        {% if delivery_method == 'pick-up' %}
          
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Pickup</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ shipping_price | money }}</strong>
  </td>
</tr>

        {% else %}
          {% if has_shipping_discount %}
  
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Shipping</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>Free</strong>
  </td>
</tr>

  <tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span class="subtotal-line__discount">
        <img src="{{ 'notifications/discounttag.png' | shopify_asset_url }}" width="18" height="18" class="discount-tag-icon" />
        <span class="subtotal-line__discount-title">{{ shipping_discount }} (-{{ shipping_amount | money }})</span>
      </span>
    </p>
  </td>
</tr>

{% else %}
  
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Shipping</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ shipping_price | money }}</strong>
  </td>
</tr>

{% endif %}

        {% endif %}

        {% if total_duties %}
          
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Duties</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ total_duties | money }}</strong>
  </td>
</tr>

        {% endif %}

        
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Taxes</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ tax_price | money }}</strong>
  </td>
</tr>


        {% if total_tip and total_tip > 0 %}
          
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Tip</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ total_tip | money }}</strong>
  </td>
</tr>

        {% endif %}
      </table>
      {% assign transaction_size = 0 %}
      {% assign transaction_amount = 0 %}
      {% for transaction in transactions %}
        {% if transaction.status == "success" %}
        {% unless transaction.kind == "authorization" or transaction.kind == "void" %}
          {% assign transaction_size = transaction_size | plus: 1 %}
          {% assign transaction_amount = transaction_amount | plus: transaction.amount %}
        {% endunless %}
        {% endif %}
      {% endfor %}
      <table class="row subtotal-table subtotal-table--total">
      {% if payment_terms and payment_terms.automatic_capture_at_fulfillment == false or b2b?%}
        {% assign due_at_date = payment_terms.next_payment.due_at | date: "%b %d, %Y" %}
        
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Total paid today</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ transaction_amount | money_with_currency }}</strong>
  </td>
</tr>

        <div class="payment-terms">
          
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Total due {{ due_at_date }}</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ payment_terms.next_payment.amount_due | money_with_currency }}</strong>
  </td>
</tr>

        </div>
      {% else %}
        
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Total</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ total_price | money_with_currency }}</strong>
  </td>
</tr>

      {% endif %}
      </table>

      {% if total_discounts > 0 %}
        <p class="total-discount">
          You saved <span class="total-discount--amount">{{ total_discounts | money }}</span>
        </p>
      {% endif %}

      {% unless payment_terms %}
      {% if transaction_size > 1 or transaction_amount < total_price %}
        <table class="row subtotal-table">
          <tr><td colspan="2" class="subtotal-table__line"></td></tr>
          <tr><td colspan="2" class="subtotal-table__small-space"></td></tr>

          {% for transaction in transactions %}
            {% if transaction.status == "success" and transaction.kind == "capture" or transaction.kind == "sale" %}
              {% if transaction.payment_details.credit_card_company %}
                {% capture transaction_name %}{{ transaction.payment_details.credit_card_company }} (ending in {{ transaction.payment_details.credit_card_last_four_digits }}){% endcapture %}
              {% else %}
                {% capture transaction_name %}{{ transaction.gateway_display_name }}{% endcapture %}
              {% endif %}

              
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>{{transaction_name}}</span>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>{{ transaction.amount | money }}</strong>
  </td>
</tr>

            {% endif %}
            {% if transaction.kind == 'refund' %}
              {% if transaction.payment_details.credit_card_company %}
                {% assign refund_method_title = transaction.payment_details.credit_card_company %}
              {% else %}
                {% assign refund_method_title = transaction.gateway %}
              {% endif %}

              
<tr class="subtotal-line">
  <td class="subtotal-line__title">
    <p>
      <span>Refund</span>
        <br>
        <small>{{ refund_method_title | capitalize }}</small>
    </p>
  </td>
  <td class="subtotal-line__value">
      <strong>- {{ transaction.amount | money }}</strong>
  </td>
</tr>

            {% endif %}
          {% endfor %}
        </table>
      {% endif %}


      {% endunless %}
    </td>
  </tr>
</table>


            </td>
          </tr>
        </table>
      </center>
    </td>
  </tr>
</table>

          <table class="row footer">
  <tr>
    <td class="footer__cell">
      <center>
        <table class="container">
          <tr>
            <td>
              
              <p class="disclaimer__subtext">If you have any questions, reply to this email or contact us at <a href="mailto:{{ shop.email }}">{{ shop.email }}</a></p>
            </td>
          </tr>
        </table>
      </center>
    </td>
  </tr>
</table>

<img src="{{ 'notifications/spacer.png' | shopify_asset_url }}" class="spacer" height="1" />

        </td>
      </tr>
    </table>
  </body>
</html>'''

In [93]:
html = html.replace('{% capture email_title %}Your order was updated {% endcapture %}', '{% capture email_title %}Seu pedido foi atualizado {% endcapture %}')
html = html.replace('Your order has been picked up{% endcapture %}', 'Seu pedido foi retirado{% endcapture %}')
html = html.replace('Thank you for shopping at {{ shop_name }}{% endcapture %}' , 'Obrigado por comprar na {{ shop_name }}{% endcapture %}')

html = html.replace('Your order is ready for pickup{% endcapture %}',
                    'Teu pedido está pronto para a recolha{% endcapture %}')

html = html.replace('Bring your confirmation email when you come to collect your order.{% endcapture %}',
                    'Traga teu email de confirmação quando vieres pegar teu pedido.{% endcapture %}')

html = html.replace('Are you ready to complete your order?{% endcapture %}', 'Estás pronto a completar teu pedido?{% endcapture %}')
html = html.replace('Thanks for stopping by <a href="{{ shop.url }}">{{ shop.name }}</a>', 'Obrigado por apareceres a <a href="{{ shop.url }}">{{ shop.name }}</a>')
html = html.replace('Complete your purchase</a></td>', 'Complete sua compra</a></td>')

html = html.replace('Your payment for {{ order.name }} has been received {% endcapture %}', 'Seu pagamento para {{ order.name }} foi recebido {% endcapture %}')

html = html.replace('Your order has been canceled', 'Seu pedido foi cancelado')

html = html.replace('Your payment couldn’t be processed for order {{ order_name }} {% endcapture %}', 
                    'Seu pagamento não pode ser processado para o pedido {{ order_name }} {% endcapture %}')
html = html.replace('You have not been charged, please try paying for the order again.', 
                    'Não foste cobrado. Tenta realizar o pagamento do pedido novamente, se faz favor.')



html = html.replace('Order {{ name }} was canceled at your request and your payment has been voided.', 
                    'O pedido {{ name }} foi cancelado por seu pedido e seu pagamento foi anulado.')
html = html.replace('Order {{ name }} was canceled because we did not have enough stock to fulfill your order and your payment has been voided.', 
                    'O pedido {{ name }} foi cancelado pois não temos itens suficientes no estoque para completar seu pedido. Seu pagamento foi anulado.')
html = html.replace('Order {{ name }} was canceled because of unforeseen circumstances and your payment has been voided.', 
                    'O pedido {{ name }} foi cancelado devido a circunstâncias imprevistas e seu pagamento foi anulado.')
html = html.replace('Order {{ name }} was canceled because your payment was declined.', 
                    'O pedido {{ name }} foi cancelado porque seu pagamento foi recusado.')
html = html.replace('Order {{ name }} was canceled because of staff error and your payment has been voided.', 
                    'O pedido {{ name }} foi cancelado devido a um erro da equipe e seu pagamento foi anulado.')

html = html.replace('Order {{ name }} was canceled at your request and your payment has been refunded.', 
                    'O pedido {{ name }} foi cancelado por seu pedido e seu pagamento foi reembolsado.')
html = html.replace('Order {{ name }} was canceled because we did not have enough stock to fulfill your order and your payment has been refunded.', 
                    'O pedido {{ name }} foi cancelado pois não temos itens suficientes no estoque para completar seu pedido. Seu pagamento foi reembolsado.')
html = html.replace('Order {{ name }} was canceled because of unforeseen circumstances and your payment has been refunded.', 
                    'O pedido {{ name }} foi cancelado devido a circunstâncias imprevistas e seu pagamento foi reembolsado.')
html = html.replace('Order {{ name }} was canceled because of staff error and your payment has been refunded.', 
                    'O pedido {{ name }} foi cancelado devido a um erro da equipe e seu pagamento foi reembolsado.')

html = html.replace('Order {{ name }} was canceled at your request and your payment has not yet been refunded.', 
                    'O pedido {{ name }} foi cancelado por seu pedido e seu pagamento ainda não foi reembolsado.')
html = html.replace('Order {{ name }} was canceled because we did not have enough stock to fulfill your order and your payment has not yet been refunded.', 
                    'O pedido {{ name }} foi cancelado pois não temos itens suficientes no estoque para completar seu pedido. Seu pagamento ainda não foi reembolsado.')
html = html.replace('Order {{ name }} was canceled because of unforeseen circumstances and your payment has not yet been refunded.', 
                    'O pedido {{ name }} foi cancelado devido a circunstâncias imprevistas e seu pagamento ainda não foi reembolsado.')
html = html.replace('Order {{ name }} was canceled because of staff error and your payment has not yet been refunded.', 
                    'O pedido {{ name }} foi cancelado devido a um erro da equipe e seu pagamento ainda não foi reembolsado.')

html = html.replace('Your order has been refunded{% endcapture %}', 
                    'Seu pedido foi reembolsado{% endcapture %}')
html = html.replace('You have received a refund{% endcapture %}', 
                    'Recebeste um reembolso{% endcapture %}')
html = html.replace('Some items in your order have been refunded{% endcapture %}', 
                    'Alguns itens do seu pedido foram reembolsados{% endcapture %}')
html = html.replace('{% capture email_body %}Total amount refunded: <strong>{{ amount | money_with_currency }}</strong>. It may take up to 10 days for this refund to appear in your account.{% endcapture %}', 
                    '{% capture email_body %}Valor total reembolsado: <strong>{{ amount | money_with_currency }}</strong>. Pode levar até 10 dias para que esse reembolso apareça na sua conta.{% endcapture %}')

html = html.replace('View your order</a></td>', 'Vê teu pedido</a></td>')
html = html.replace('Visit our store</a></td>', 'Visite nossa loja</a></td>')
html = html.replace('<p style="color:#000">Come back or visit <a href="{{ shop.url }}">our online store</a>.</p>',
                    '<p style="color:#000">Volte ou visite <a href="{{ shop.url }}">nossa loja online</a>.</p>')

html = html.replace('Thank you for your purchase!', 'Obrigado pela sua compra!')
html = html.replace('Open map&nbsp;', 'Abrir Mapa&nbsp;')
    
html = html.replace('<h3>Order summary</h3>', 
                    '<h3>Resumo do Pedido</h3>')

html = html.replace('Your order has been refunded{% endcapture %}', 
                    'Seu pedido foi reembolsado{% endcapture %}')
html = html.replace('Your order has been refunded{% endcapture %}', 
                    'Seu pedido foi reembolsado{% endcapture %}')
html = html.replace('Your order has been refunded{% endcapture %}', 
                    'Seu pedido foi reembolsado{% endcapture %}')
html = html.replace('Your order has been refunded{% endcapture %}', 
                    'Seu pedido foi reembolsado{% endcapture %}')

html = html.replace('<h3>Updated order</h3>', '<h3>Pedido Atualizado</h3>')
html = html.replace('Invoice {{ order_name }}', 'Fatura {{ order_name }}')
html = html.replace('<h3>Gift card</h3>', '<h3>Cartão Presente</h3>')
html = html.replace('<p> You’ll receive separate emails for any gift cards.</p>', '<p> Receberás e-mails separados para todos os cartões-presente.</p>')

html = html.replace('class="button__text">Pay now</a></td>', 'class="button__text">Pague Agora</a></td>')
html = html.replace('Order {{ order_name }}', 'Pedido {{ order_name }}')
html = html.replace('PO number #{{ po_number }}', 'Nº do Pedido #{{ po_number }}')
html = html.replace('<h3>Removed Items</h3>', '<h3>Itens Removidos</h3>')
html = html.replace('<span class="order-list__item-variant">Part of: {{ group.display_title }}</span><br/>', '<span class="order-list__item-variant">Parte de: {{ group.display_title }}</span><br/>')
html = html.replace('Free', 'Grátis')
html = html.replace('<span>Updated total</span>', '<span>Total Atualizado</span>')
html = html.replace('<span>Already paid</span>', '<span>Pagamentos Realizados</span>')
html = html.replace('<span>Amount to pay</span>', '<span>Restante a ser pago</span>')

html = html.replace('<span>Order Discount</span>', '<span>Desconto</span>')
html = html.replace('<span>Order Discounts</span>', '<span>Descontos</span>')
html = html.replace('<span>Pickup</span>', '<span>Recolha</span>')
html = html.replace('<span>Shipping</span>', '<span>Entrega</span>')
html = html.replace('<span>Duties</span>', '<span>Obrigações</span>')
html = html.replace('<span>Taxes</span>', '<span>Taxos</span>')
html = html.replace('<span>Tip</span>', '<span>Gorjeta</span>')
html = html.replace('<span>Total paid today</span>', '<span>Total pago hoje</span>')
html = html.replace('<span>Total due {{ due_at_date }}</span>', '<span>Total até {{ due_at_date }}</span>')
html = html.replace('You saved <span class="total-discount--amount">{{ total_discounts | money }}</span>', 'Economizaste <span class="total-discount--amount">{{ total_discounts | money }}</span>')

html = html.replace('<h3>Customer information</h3>', '<h3>Informação do Cliente</h3>')
html = html.replace('<h4>Shipping address</h4>', '<h4>Endereço de Entrega</h4>')
html = html.replace('<h4>Billing address</h4>', '<h4>Endereço de Cobrança</h4>')
html = html.replace('<h4>Location</h4>', '<h4>Localidade</h4>')
html = html.replace("Due {{ due_date | date: format: 'date' }}", "Até {{ due_date | date: format: 'date' }}")
html = html.replace('Gift card balance: {{ transaction', 'Saldo do Cartão Presente: {{ transaction')
html = html.replace('<span>Estimated taxes</span>', '<span>Taxos Estimados</span>')

html = html.replace('>Refunded<', '>Reembolsado<')
html = html.replace('<span>Refund</span>', '><span>Reembolso</span><')
html = html.replace('<h2>Payment of {{ order.total_outstanding', '<h2>Pagamento de {{ order.total_outstanding')
html = html.replace('Payment', 'Pagamento')
html = html.replace('Shipping method', 'Método de Entrega')
html = html.replace('If you have any questions, reply to this email or contact us at ', 'Se tiveres alguma dúvida, respondas a este e-mail ou entre em contato connosco pelo e-mail ')

html = html.replace('<span>Balance due</span>','<span>Saldo Devedor</span>')

In [94]:
with open('text.html', 'w') as f:
    f.write(html)